### Create the right environment for Feel ++


In [ ]:
import os
import sys
import feelpp
import feelpp.toolboxes.core as tb
from tools.GeoToMsh import geo_to_msh
from tools.GmeshRead import mesh2d
from tools.cmap import custom_cmap
import pandas as pd
import numpy as np
import pyvista as pv

import plotly.express as px
from plotly.subplots import make_subplots
import itertools
import torch
from pathlib import Path
from tools.Poisson_feelpp import Poisson_feel

# mandatory things
sys.argv = ["feelpp_app"]
e = feelpp.Environment(sys.argv,
                       opts=tb.toolboxes_options("coefficient-form-pdes", "cfpdes"),
                       config=feelpp.localRepository('feelpp_cfpde'))

# ------------------------------------------------------------------------- #
# Poisson problem
# - div (diff * grad (u)) = f    in Omega
#                     u   = g    in Gamma_D
# Omega = domain, either cube or ball
# Approx = lagrange Pk of order order
# mesh of size h



### Solving the Laplacian problem

In [ ]:
# Solving 
P = Poisson_feel(dim = 2)

# for square domain
u_exact = 'sin(2*pi*x) * sin(2*pi*y)'
rhs = '8*pi*pi*sin(2*pi*x) * sin(2*pi*y)'
P(rhs=rhs, g='0', order=1, u_exact = u_exact)



### Extracting the solution values

In [ ]:
import pyvista as pv
import pandas as pd

import os

# Fichier .case
file_path = 'cfpdes-2d-p1.exports/Export.case'
data = pv.read(file_path)

# Extraire les données de chaque bloc
for i, block in enumerate(data):
    if block is None:
        continue
    print(f"Bloc {i}:")
    print(block)

    print("Champs de points disponibles:", block.point_data.keys())
    print("Champs de cellules disponibles:", block.cell_data.keys())

    solution = 'cfpdes.poisson.u'

    solution_expression = block.point_data[solution]
    print("Expression de la fonction extraite:", solution_expression)

    df = pd.DataFrame(block.point_data)
    print(df.head())
    
poisson_u = block.point_data['cfpdes.poisson.u']
print("Valeurs de 'cfpdes.poisson.u':")
print(poisson_u) 

### comparing solution extracted and solution computed

In [ ]:
from xvfbwrapper import Xvfb
import pyvista as pv 
import matplotlib.pyplot as plt


vdisplay = Xvfb()
vdisplay.start()
pv.set_jupyter_backend('static') 
#pv.start_xvfb()
def pv_get_mesh(mesh_path):
    reader = pv.get_reader(mesh_path)
    mesh = reader.read()
    return mesh

def pv_plot(mesh, field, clim=None, cmap=custom_cmap, cpos='xy', show_scalar_bar=True, show_edges=True):
    mesh.plot(scalars=field, clim=clim, cmap=cmap, cpos=cpos, show_scalar_bar=show_scalar_bar, show_edges=show_edges)

def myplots(dim=2, field="cfpdes.poisson.u", factor=1, cmap=custom_cmap):
    mesh = pv_get_mesh((f"cfpdes-{P.dim}d-p{P.order}.exports/Export.case"))
    #pv_plot(mesh, field)
    pl = pv.Plotter(shape=(1,2))

    pl.add_title(f'Solution P{P.order}', font_size=18)
    pl.add_mesh(mesh[0], scalars = "cfpdes.poisson.u", cmap=custom_cmap)

    pl.subplot(0,1)
    pl.add_title('u_extracted ' , font_size=10)
    pl.add_mesh(mesh[0].copy(), scalars = poisson_u , cmap=custom_cmap)

    
    pl.link_views()
    pl.view_xy()
    pl.show()
    #pl.screenshot(plot)

myplots(dim=2,factor= 1)

### Reading mesh nodes

In [ ]:
# Reading nodes for square mesh
mesh = 'omega-2.msh'
my_mesh = mesh2d(mesh)
my_mesh.read_mesh()
coordinates = my_mesh.Nodes
for i in range(my_mesh.Nnodes):
    print(f'\nu({coordinates[i]}) = ', poisson_u[i] )



In [ ]:
# for disk domain
u_exact =  'sin(pi*(x*x + y*y))'
rhs = '4* pi * (-cos(pi* (x*x + y*y)) + pi * (x*x + y*y)* sin(pi* (x*x + y*y)))'
P(rhs=rhs, g='0', order=1, geofile='disk.geo', u_exact = u_exact)

# Fichier .case
file_path = 'cfpdes-2d-p1.exports/Export.case'
data = pv.read(file_path)

# Extraire les données de chaque bloc
for i, block in enumerate(data):
    if block is None:
        continue
    print(f"Bloc {i}:")
    print(block)

    print("Champs de points disponibles:", block.point_data.keys())
    print("Champs de cellules disponibles:", block.cell_data.keys())

    solution = 'cfpdes.poisson.u'

    solution_expression = block.point_data[solution]
    print("Expression de la fonction extraite:", solution_expression)

    df = pd.DataFrame(block.point_data)
    print(df.head())
    
poisson_u = block.point_data['cfpdes.poisson.u']
print("Valeurs de 'cfpdes.poisson.u':")
print(poisson_u) 

from xvfbwrapper import Xvfb
import pyvista as pv 
import matplotlib.pyplot as plt


vdisplay = Xvfb()
vdisplay.start()
pv.set_jupyter_backend('static') 
#pv.start_xvfb()
def pv_get_mesh(mesh_path):
    reader = pv.get_reader(mesh_path)
    mesh = reader.read()
    return mesh

def pv_plot(mesh, field, clim=None, cmap=custom_cmap, cpos='xy', show_scalar_bar=True, show_edges=True):
    mesh.plot(scalars=field, clim=clim, cmap=cmap, cpos=cpos, show_scalar_bar=show_scalar_bar, show_edges=show_edges)

def myplots(dim=2, field="cfpdes.poisson.u", factor=1, cmap=custom_cmap):
    mesh = pv_get_mesh((f"cfpdes-{P.dim}d-p{P.order}.exports/Export.case"))
    #pv_plot(mesh, field)
    pl = pv.Plotter(shape=(1,2))

    pl.add_title(f'Solution P{P.order}', font_size=18)
    pl.add_mesh(mesh[0], scalars = "cfpdes.poisson.u", cmap=custom_cmap)

    pl.subplot(0,1)
    pl.add_title('u_extracted ' , font_size=10)
    pl.add_mesh(mesh[0].copy(), scalars = poisson_u , cmap=custom_cmap)

    
    pl.link_views()
    pl.view_xy()
    pl.show()
    #pl.screenshot(plot)

myplots(dim=2,factor= 1)

In [ ]:

# Reading nodes from disk mesh
mesh = 'disk.msh'
my_mesh = mesh2d(mesh)
my_mesh.read_mesh()
coordinates = my_mesh.Nodes
for i in range(my_mesh.Nnodes):
    print(f'\nu({coordinates[i]}) = ', poisson_u[i] )

### Computing error convergence rates

In [ ]:
def runLaplacianPk(P, df, model, fn = f'omega-{P.dim}.geo', verbose=False):
    """Generate the Pk case"""
    meas = dict()
    dim, order, json = model    
    for h in df['h']:
      m = P.feel_solver(filename=fn, h=h, json=json, dim=dim, verbose=verbose)
    for norm in ['L2', 'H1']:
      meas.setdefault(f'P{order}-Norm_poisson_{norm}-error', [])
      meas[f'P{order}-Norm_poisson_{norm}-error'].append(m.pop(f'poisson_{norm}-error'))
    df = df.assign(**meas)
    for norm in ['L2', 'H1']:
      df[f'P{order}-poisson_{norm}-convergence-rate']=np.log2(df[f'P{order}-Norm_poisson_{norm}-error'].shift() / df[f'P{order}-Norm_poisson_{norm}-error']) / np.log2(df['h'].shift() / df['h'])

    return df



def runConvergenceAnalysis(P, json,dim=2,hs=[0.1,0.05,0.025],orders=[1,2],verbose=False):
  df=pd.DataFrame({'h':hs})
  for order in orders:
    df=runLaplacianPk(df=df,model=[dim,order,json(dim=dim,order=order)],verbose=verbose)
  print('df = ', df.to_markdown())
  return df

def plot_convergence(P, df,dim,orders=[1,2]):
  fig=px.line(df, x="h", y=[f'P{order}-Norm_poisson_{norm}-error' for order,norm in list(itertools.product(orders,['L2','H1']))])
  fig.update_xaxes(title_text="h",type="log")
  fig.update_yaxes(title_text="Error",type="log")
  for order,norm in list(itertools.product(orders,['L2','H1'])):
    fig.update_traces(name=f'P{order} - {norm} error - {df[f"P{order}-poisson_{norm}-convergence-rate"].iloc[-1]:.2f}', selector=dict(name=f'P{order}-Norm_poisson_{norm}-error'))
  fig.update_layout(
          title=f"Convergence rate for the {dim}D Laplacian problem",
          autosize=False,
          width=900,
          height=900,
      )
  return fig


In [ ]:

u_exact = 'sin(2*pi*x) * sin(2*pi*y)'
rhs = '8*pi*pi*sin(2*pi*x) * sin(2*pi*y)'

P(rhs=rhs, g='0', order=1, u_exact = u_exact)
poisson_json = lambda order,dim=2,name="u": P.model

with open(f'poisson-{P.dim}d.json', 'w') as f:
      import json
      f.write(json.dumps(poisson_json(dim=P.dim,order=1),indent=1))

json=poisson_json

model=[P.dim,P.order,json(dim=P.dim,order=P.order)]
print(model)
hs=[0.1,0.05,0.025,0.0125]
df=pd.DataFrame({'h':hs})

"""
df= runLaplacianPk(P, df=df, model=model, verbose=True)
df= runConvergenceAnalysis( P, json=json,dim=2,verbose=True)
"""

In [ ]:
fig= plot_convergence(P, df,dim=2)
fig.show()

